# FedLib: Simulating FedAvg using FedLib virtual Federated environment

## Importing supportive libaries
This notebook shows a demo on PyTorch back-end model impelementation.

In the very begining, we import the supporting libraries.

In [1]:
import torch
import numpy as np
import copy

from fedlib.simulator import BaseSimulator as Simulator
from fedlib.lib import Server, Client
from fedlib.networks import resnet20
from fedlib.lib.sampler import random_sampler
from fedlib.lib.algo.fedavg import Trainer 
from fedlib.datasets import partition_data, get_dataloader

/Users/sixingyu/opt/anaconda3/envs/nas/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Define arguments
Here we define arguments. To show an intuitive example, we show the demo store all the parameters in a dictionary in the following code block.
We also provide APIs for you create your arguments in a `*.yaml` file.

In [2]:
args = {}
args["n_clients"] = 100
args["device"] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args['sample_fn'] = random_sampler
args['trainer'] = Trainer()
args['communicator'] = None
args["test_dataset"] = None
args["partition"] = "noniid-labeldir"
args["dataset"] = "cifar10"
args["datadir"] = "./data"
args["beta"] = 0.5
args["batch_size"] = 64
args["global_model"] = resnet20()
args["lr"] = 0.01
args["optimizer"] = "SGD"
args["lr_scheduler"] = "ExponentialLR"

In [3]:
X_train, y_train, X_test, y_test, net_dataidx_map, traindata_cls_counts = partition_data(
    args["dataset"], args["datadir"], args['partition'], args['n_clients'], beta=args['beta'])
n_classes = len(np.unique(y_train))
train_dl_global, test_dl_global, train_ds_global, test_ds_global = get_dataloader(args["dataset"],
                                                                                    args["datadir"],
                                                                                      args["batch_size"],
                                                                                      32)
args["test_dataset"] = test_dl_global



Files already downloaded and verified
Files already downloaded and verified


INFO:root:Data statistics: {0: {0: 20, 1: 24, 2: 40, 3: 69, 4: 98, 5: 10, 7: 108, 8: 68, 9: 39}, 1: {0: 80, 1: 111, 2: 95, 3: 3, 4: 121, 5: 17, 6: 1, 8: 185}, 2: {0: 3, 1: 84, 2: 14, 3: 69, 4: 4, 5: 51, 6: 18, 7: 1, 8: 145, 9: 28}, 3: {0: 169, 1: 47, 2: 38, 3: 2, 4: 42, 5: 2, 6: 1, 7: 50, 9: 8}, 4: {0: 45, 1: 32, 2: 10, 3: 4, 4: 12, 5: 109, 6: 9, 7: 6, 8: 19, 9: 12}, 5: {0: 138, 1: 4, 2: 15, 3: 25, 4: 15, 5: 112, 6: 1, 7: 8, 8: 49, 9: 33}, 6: {0: 5, 1: 1, 2: 84, 3: 31, 4: 61, 5: 2, 6: 6, 7: 19, 8: 74, 9: 13}, 7: {0: 83, 1: 10, 2: 8, 3: 5, 4: 48, 5: 81, 6: 9, 7: 18, 8: 6, 9: 22}, 8: {0: 1, 1: 1, 2: 41, 3: 39, 4: 74, 5: 18, 6: 83, 7: 1, 8: 63, 9: 38}, 9: {0: 142, 1: 23, 2: 19, 3: 91, 4: 2, 5: 178, 6: 27, 7: 3, 8: 86}, 10: {0: 73, 1: 11, 2: 30, 3: 7, 5: 39, 6: 3, 7: 2, 8: 17, 9: 2}, 11: {0: 8, 2: 93, 3: 37, 4: 12, 5: 19, 6: 7, 7: 128, 8: 13}, 12: {0: 4, 1: 9, 2: 70, 3: 8, 4: 1, 5: 27, 7: 90, 8: 10, 9: 5}, 13: {0: 3, 1: 1, 2: 112, 3: 129, 4: 155, 5: 15, 6: 25, 7: 344}, 14: {0: 131, 1: 45, 

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
server = Server(**args)
clients = {}
for id in range(args["n_clients"]):
    dataidxs = net_dataidx_map[id]
    args["id"] = id
    args["trainloader"], _, _, _ = get_dataloader(args["dataset"], args["datadir"], args['batch_size'], 32, dataidxs)
    args["model"] = copy.deepcopy(args["global_model"])
    clients[id] = Client(**args)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files alread

In [5]:
criterion = torch.nn.CrossEntropyLoss()


In [6]:
simulator = Simulator(server=server, clients=clients, communication_rounds=10,n_clients= 100,sample_rate=.1)

In [7]:
simulator.run(criterion,2)

INFO:root:*******starting rounds 1 optimization******
INFO:root:optimize the 1-th clients
INFO:root:Update Epoch: 0 	Loss: 2.701399
INFO:root:Epoch: 0	Loss: 2.137350
INFO:root:Update Epoch: 1 	Loss: 1.860873
INFO:root:Epoch: 1	Loss: 1.928999
INFO:root:optimize the 60-th clients
INFO:root:Update Epoch: 0 	Loss: 3.288010
INFO:root:Epoch: 0	Loss: 2.381786
INFO:root:Update Epoch: 1 	Loss: 1.799584
INFO:root:Epoch: 1	Loss: 2.034661
INFO:root:optimize the 88-th clients
INFO:root:Update Epoch: 0 	Loss: 4.700487
INFO:root:Update Epoch: 0 	Loss: 0.842412
INFO:root:Epoch: 0	Loss: 1.605391
INFO:root:Update Epoch: 1 	Loss: 0.663421
INFO:root:Update Epoch: 1 	Loss: 0.825380
INFO:root:Epoch: 1	Loss: 1.135946
INFO:root:optimize the 4-th clients
INFO:root:Update Epoch: 0 	Loss: 3.369145
INFO:root:Epoch: 0	Loss: 2.863790
INFO:root:Update Epoch: 1 	Loss: 2.283384
INFO:root:Epoch: 1	Loss: 2.462654
INFO:root:optimize the 69-th clients
INFO:root:Update Epoch: 0 	Loss: 4.018911
INFO:root:Epoch: 0	Loss: 2.66

TypeError: aggregate() takes 1 positional argument but 2 were given